In [1]:
import numpy as np
import os
import tensorflow as tf

n_inputs = 3 # 입력데이터
n_neurons = 5 # 셀의 가중치 사이즈
tf.reset_default_graph() # 그래프 초기화(변수 생성)

X0 = tf.placeholder(tf.float32, [None, n_inputs]) # 4x3 만단데이터 개수가 3
X1 = tf.placeholder(tf.float32, [None, n_inputs]) # 4x3
# [0,1,2] 사이즈가 하나의 셀로 입력
# FFNN(feed foweard neural network)

# 가중치 사이즈 => 특성을 찾아내는 것
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)

# static_rnn : rnn network : 4개의 셀이 자동으로 연결되면서 메모리 확보
output_seqs, states = tf.contrib.rnn.static_rnn(basic_cell, [X0,X1], dtype=tf.float32)

Y0,Y1 = output_seqs

init = tf.global_variables_initializer()
# 출력, 다음셀로 전달되는 값 : 마지막 states값(수평으로 셀을 연결)
X0_batch = np.array([[0,1,2],[3,4,5],[6,7,8],[9,0,1]]) # 4x3
X1_batch = np.array([[9,8,7],[0,0,0],[6,5,4],[3,2,1]])

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [2]:
with tf.Session() as sess:
    init.run()
    Y0_val, Y1_val = sess.run([Y0,Y1], feed_dict={X0:X0_batch, X1:X1_batch})
    
    print('처음 데이터 특성 : \n{} \n차수 : {}\n'.format(Y0_val, Y0_val.shape))
    print('두번째 데이터 특성 : \n{} \n차수 : {}'.format(Y1_val, Y1_val.shape))
    # 4x3 => 4x5

처음 데이터 특성 : 
[[ 0.0523796   0.13707756  0.1170179  -0.8207902   0.70326054]
 [ 0.95273715  0.6465866  -0.9544191  -0.9975975   0.99701744]
 [ 0.9986997   0.88554066 -0.99914044 -0.9999706   0.99997437]
 [ 0.9950511   0.2547385  -0.99997675  0.4955649   0.73329765]] 
차수 : (4, 5)

두번째 데이터 특성 : 
[[ 0.9999667   0.9330205  -0.99999547 -0.99997157  0.9999919 ]
 [ 0.29858527 -0.3200989  -0.78226167 -0.611944   -0.6193791 ]
 [ 0.99913895  0.7305888  -0.99991745 -0.99907273  0.9957389 ]
 [ 0.94642884 -0.05927787 -0.98769766 -0.93706256  0.8519384 ]] 
차수 : (4, 5)


? 셀이 가지고있느 가중치 사이즈 : 3x5??
왜 5개라고 정해진걸까? => 인풋과 뉴런수에의해 결정된거같다.

# 분류

In [3]:
tf.reset_default_graph() # 그래프 초기화
n_steps = 28 # 셀수 
n_inputs = 28 # 셀당 인풋사이즈
n_neurons = 150 # 누런수
n_outputs = 10 # 확률사이즈
learning_rate = 0.001

# 3차원으로 데이터를 받아오기 때문에
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs]) # ?x28x28
Y = tf.placeholder(tf.int32, [None])

# FFNN
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units = n_neurons) #150
# 데이터가 3차원으로 입력됨
# 2차원이 셀수
# 셀 : 28개
# state 마지막셀의 수평으로 전달되는 값
# state마지막셀의 output과 같다.
# 28개의 셀이 있는데 마지막 한개의 output을 사용 => many to one
# 감정분류 -> 분류
# state의 차수 : 150x150
# output의 차수 : latent time 지연시간을 통해 계산된셀의
# 모든 값을 결합 출력 150x28x150
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)
# outputs.shape = > 150x28x150
# states.shape => 150x150

# 150개의 특징중에 10개만 추출
# dense 입력차수, 출력차수만 지정하면 자동으로 바이어스를 생성
# 가중치 공간을 확보해서
# 150x150, => 150x10
# dense의 가중치 사이즈 = 150x10
logits = tf.layers.dense(states, n_outputs)
# 분류를 위한 미분이 가능한 식으로 바뀜
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = Y, logits=logits)
# 배치 사이즈 => 평균을 통해서 loss를 구한다.
loss = tf.reduce_mean(xentropy)

# momentum, propgrad
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
training_op = optimizer.minimize(loss)

# 양쪽에서 큰놈으로
# 가장큰값
correct = tf.nn.in_top_k(logits, Y,1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
init = tf.global_variables_initializer()

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Use keras.layers.dense instead.


In [4]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('/tmp/data/')
# 원래이미지 모양으로 학습
X_test = mnist.test.images.reshape((-1,n_steps,n_inputs))
Y_test = mnist.test.labels

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [5]:
n_epochs = 10
batch_size = 150

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        # 60000/150 을 10번 반복해서 돈다.
        for iteration in range(mnist.train.num_examples // batch_size): 
            X_batch, Y_batch = mnist.train.next_batch(batch_size)
            X_batch = X_batch.reshape((-1,n_steps, n_inputs))
            sess.run(training_op, feed_dict={X:X_batch,Y:Y_batch})
        acc_train = accuracy.eval(feed_dict={X:X_batch,Y:Y_batch})
        acc_test = accuracy.eval(feed_dict = {X:X_test, Y:Y_test})
        print('epoch {} Train accuracy : {} Test accuracy : {}'.format(epoch, acc_train, acc_test))

epoch 0 Train accuracy : 0.9399999976158142 Test accuracy : 0.9380000233650208
epoch 1 Train accuracy : 0.9333333373069763 Test accuracy : 0.9470000267028809
epoch 2 Train accuracy : 0.9800000190734863 Test accuracy : 0.9584000110626221
epoch 3 Train accuracy : 0.9666666388511658 Test accuracy : 0.9664999842643738
epoch 4 Train accuracy : 0.9733333587646484 Test accuracy : 0.9649999737739563
epoch 5 Train accuracy : 0.9666666388511658 Test accuracy : 0.9639000296592712
epoch 6 Train accuracy : 0.9800000190734863 Test accuracy : 0.9670000076293945
epoch 7 Train accuracy : 0.9733333587646484 Test accuracy : 0.9700000286102295
epoch 8 Train accuracy : 0.9800000190734863 Test accuracy : 0.9668999910354614
epoch 9 Train accuracy : 0.9866666793823242 Test accuracy : 0.9710999727249146


## multi rnn cell
- MultiRNNCell : 3단계로 구성되어있다.(basic rnn cell, multi rnn cell, dynamic_rnn으로 수성)
- BasicRNNCell : 기본데이터가 입력되는 곳은 basic rnn cell 이다.!!!
- MultiRNNCell : 수직으로 레이어 구성(아까는 이거 없었음) 수평으로 구성되어있었는데 수직으로 구성 >>밑에보면 3개로 -> multi rnn cell로 의해서 수직으로 셀이 구성
- dynamic_rnn : 이것이 모델이다.

In [6]:
tf.reset_default_graph()
n_steps = 28
n_inputs=28
n_outputs = 10
learning_rate = 0.001

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
Y = tf.placeholder(tf.int32, [None])

n_neurons = 100
n_layers = 3 # 3개의 멀티레이어

# basic rnn cell은 3개만들어짐 for문을 통해 확인가능 -> 3개의 셀 생성
layers = [tf.contrib.rnn.BasicRNNCell(num_units = n_neurons, activation= tf.nn.relu) for layer in range(n_layers)]
multi_layer_cell = tf.contrib.rnn.MultiRNNCell(layers) # 3개의 셀을 조합해서 

Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.


- 이번엔 dynamic_rnn이다. : 셀로 입력되는 데이터의 개수에 맞추어서 셀을 구성할 수 있다.
- ex) 나는 학교에 간다.(2글자 + 3글자 + 2글자 -> 사이즈가 다르다. -> 사이즈가 다르면 static_rnn에서 동일한 사이즈로 맞췄다 -> 큰것을 기준으로 작은것을 padding했었다. -> 결과가 안좋게 나오기 때문에 dynamic_rnn을 쓴다.)
- dynamic_rnn은 사이즈가 다르면 다른데로 상관하지 않는다.(입력사이즈를 변동 -> 가중치를 조절한다. -> 나가는 특징은 일치함 = neural수는 같다.)

In [7]:
# 3층 셀이 28개가 조성이 된다. -> 고정사이즈
outputs, states = tf.nn.dynamic_rnn(multi_layer_cell, X, dtype=tf.float32)

# states가 몇개가 발생하는가?
# 3x150x100(층이 3층이므로 3, 배치사이즈가 150, 150x100에 3층이므로 3x150x100)
# concat : 열방향으로 하게됨(axis=1에 의해서) -> 150x300이 된다.
states_concat = tf.concat(axis = 1, values=states)
# dense : 입력차수와 출력차수를 넣어주면 가중치 사이즈를 자기가 만든다.
# 150x300이 들어오고, 출력 차수가 10이니까 가중치는 300x10이됨
logits=tf.layers.dense(states_concat, n_outputs)

# 가충치계산해서 나온결과는 150(미니배치 사이즈)x10(확률적으로 나타나는 10개의 특징 = 확률값))
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=Y,  logits=logits)
loss = tf.reduce_mean(xentropy)

optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
training_op = optimizer.minimize(loss)

correct = tf.nn.in_top_k(logits,Y,1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()

In [ ]:
n_epoch = 10
batch_size = 150
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, Y_batch = mnist.train.next_batch(batch_size)
            # 이미지 사이즈로 생성
            X_batch = X_batch.reshape((-1,n_steps,n_inputs))
            sess.run(training_op, feed_dict={X:X_batch, Y:Y_batch})
        acc_train = accuracy.eval(feed_dict={X:X_batch, Y:Y_batch})
        acc_test = accuracy.eval(feed_dict={X:X_test, Y:Y_test})
        print(epoch, 'Train accuracy : ', acc_train, 'Test accyract : ',acc_test)

0 Train accuracy :  0.97333336 Test accyract :  0.9383
1 Train accuracy :  0.97333336 Test accyract :  0.963
2 Train accuracy :  0.97333336 Test accyract :  0.9686
3 Train accuracy :  0.94 Test accyract :  0.9739


In [ ]:
import tensorflow as tf
from tensorflow.python.framework import ops
ops.reset_default_graph()
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('./mnist/data/', one_hot=True)

learning_rate = 0.001
total_epoch = 10
batch_size = 128
n_input = 28
n_step = 28
n_hidden = 128
n_class = 10

X = tf.placeholder(tf.float32, [None, n_step, n_input])
Y = tf.placeholder(tf.float32, [None, n_class])
W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

cell =   